In [1]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [2]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
assets = [:USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net    │
├────────────┼────────────┼──────────────┼─────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │   0.0246253 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │   0.0560778 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │   0.0302388 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │   0.0125024 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │  -0.0201209 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │ -0.00119203 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │     0.01818 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │  -0.0566908 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │   -0.042229 │
│

In [3]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))

417×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2022-09-30
┌────────────┬─────────────┬──────────────┬─────────────┐
│            │ USA_Net     │ World_ex_USA │ EM_Net      │
├────────────┼─────────────┼──────────────┼─────────────┤
│ 1988-01-31 │   0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │   0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │  -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │  0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │  0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │   0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │  -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │   -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │      ⋮      │      ⋮       │      ⋮      │
│ 2022-03-31 │     0.03417 │    0.0115373 │  -0.0228443 │
│ 2022-04-30 │  -0.0952756 │   -0.0679247 │  -0.0572341 │
│ 2022-05-31 │ -0.00270982 │    0.0082682 │   0.0043937 │
│ 2022-06-30 │  -0.0868359 │   -0.0988188 │  -0.068761

In [4]:
[assets; :ACWI_Net]

4-element Vector{Symbol}:
 :USA_Net
 :World_ex_USA
 :EM_Net
 :ACWI_Net

In [5]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1507,1.246,0.0676,0.4027,-2.257,-0.84,-0.2075,0.0703,1.6368,0.4485,0.1412
World_ex_USA,0.1653,1.4555,0.0499,0.4126,-2.7986,-0.913,-0.2363,0.1497,2.4191,0.3017,0.1038
EM_Net,0.233,1.893,0.0684,0.5702,-4.1,-1.0973,-0.3213,0.1423,3.896,0.2936,0.1038


In [6]:
hmm = hmm_est(period_returns, 3)
regime_summary(hmm[1], assets, 12)


Means
Standard deviations
Regimes transition matrix


Regime,USA_Net,World_ex_USA,EM_Net
1,0.111,0.073,0.152
2,-0.031,-0.075,-0.058
3,0.323,0.485,0.301


Regime,USA_Net,World_ex_USA,EM_Net
1,0.080,0.091,0.156
2,0.195,0.209,0.295
3,0.074,0.057,0.085


"Correlations in regime 1"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.702,0.582
World_ex_USA,0.702,1.000,0.887
EM_Net,0.582,0.887,1.000


"Correlations in regime 2"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.888,0.796
World_ex_USA,0.888,1.000,0.846
EM_Net,0.796,0.846,1.000


"Correlations in regime 3"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.777,0.578
World_ex_USA,0.777,1.000,0.753
EM_Net,0.578,0.753,1.000


Regime,1,2,3
1,0.713,0.063,0.224
2,0.000,0.941,0.059
3,1.000,0.000,0.000


In [7]:
n_assets = length(assets)
scenarios = simulate_hmm(hmm[1], n_assets, 180, 100_000)

for a in 1:n_assets
    print_percentiles(scenarios[a,:,:], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12, string(assets[a]))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.3831,-0.3419,-0.3295,-0.328,-0.3226,-0.3224,-0.3245,-0.3207,-0.3252,-0.3248,-0.3258,-0.3251,-0.326,-0.3255,-0.3264
0.05,-0.3008,-0.2531,-0.237,-0.2344,-0.231,-0.2321,-0.2332,-0.2324,-0.2336,-0.2328,-0.2316,-0.2345,-0.2332,-0.2362,-0.2323
0.25,-0.101,-0.0405,-0.0211,-0.017,-0.0155,-0.0158,-0.0169,-0.0189,-0.0172,-0.0171,-0.0186,-0.0188,-0.0175,-0.0176,-0.0163
0.5,0.0294,0.0846,0.0944,0.0983,0.0985,0.0973,0.0983,0.0967,0.0977,0.0976,0.0971,0.0977,0.0978,0.0979,0.0983
0.75,0.1475,0.1773,0.1817,0.1836,0.1828,0.1834,0.1829,0.1816,0.1841,0.1826,0.1832,0.183,0.183,0.183,0.1835
0.95,0.3033,0.2981,0.2961,0.297,0.2972,0.2971,0.2988,0.2948,0.2965,0.2977,0.2975,0.2972,0.297,0.2968,0.2975
0.98,0.3704,0.3548,0.3501,0.3517,0.3503,0.3511,0.3519,0.3487,0.3511,0.3514,0.3513,0.3518,0.3496,0.3507,0.3513


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.4506,-0.4074,-0.3957,-0.3899,-0.3848,-0.3841,-0.388,-0.3825,-0.3866,-0.3879,-0.3878,-0.3897,-0.3863,-0.3881,-0.3887
0.05,-0.362,-0.3098,-0.2929,-0.2889,-0.2856,-0.2841,-0.2887,-0.2826,-0.2872,-0.2879,-0.2867,-0.2895,-0.2885,-0.2881,-0.2863
0.25,-0.1423,-0.0745,-0.0528,-0.0473,-0.046,-0.0466,-0.0478,-0.0487,-0.047,-0.0471,-0.0482,-0.0487,-0.0468,-0.0474,-0.0473
0.5,0.0029,0.0692,0.0812,0.0858,0.0855,0.0855,0.0857,0.0844,0.0858,0.085,0.0846,0.0845,0.0852,0.0852,0.0855
0.75,0.1346,0.176,0.1811,0.1843,0.1832,0.1837,0.1835,0.1825,0.1839,0.183,0.183,0.183,0.1824,0.1828,0.1832
0.95,0.303,0.3054,0.3051,0.3055,0.3056,0.3051,0.3071,0.3051,0.3062,0.3058,0.3068,0.3057,0.3064,0.3051,0.307
0.98,0.3723,0.3633,0.3588,0.3607,0.3602,0.358,0.3641,0.3588,0.3598,0.3595,0.3611,0.3611,0.3604,0.3577,0.3608


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.5982,-0.5369,-0.519,-0.5092,-0.5152,-0.5079,-0.508,-0.5065,-0.5122,-0.5094,-0.5067,-0.5131,-0.5123,-0.5069,-0.5108
0.05,-0.4718,-0.401,-0.3811,-0.3769,-0.3765,-0.3711,-0.3747,-0.3726,-0.3755,-0.3751,-0.3735,-0.3791,-0.3719,-0.3745,-0.3747
0.25,-0.1774,-0.0936,-0.0712,-0.0652,-0.0642,-0.0639,-0.0665,-0.0657,-0.0639,-0.065,-0.0659,-0.0678,-0.0651,-0.0651,-0.0644
0.5,0.0162,0.0849,0.0981,0.1029,0.1037,0.1037,0.1038,0.1017,0.1027,0.1036,0.1028,0.1011,0.1024,0.1024,0.1027
0.75,0.1931,0.2327,0.2393,0.2427,0.2427,0.2423,0.2424,0.2411,0.2415,0.2418,0.2414,0.2413,0.2411,0.2411,0.2411
0.95,0.4364,0.431,0.4305,0.4318,0.4314,0.4297,0.433,0.4286,0.431,0.4302,0.432,0.4302,0.4319,0.4296,0.4331
0.98,0.5397,0.523,0.5186,0.5192,0.5166,0.5134,0.5196,0.5145,0.5196,0.5171,0.5179,0.5144,0.5173,0.5178,0.5212


In [19]:
y_data = annualise(scenarios[1, :, 25:36],12)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

"mean"

0.07002120044189371

"std"

0.163224694079844

"skewness"

-0.686740153265794

"kurtosis"

0.7693066012745633

In [9]:
weights = [0.321, 0.67, 0.009]
#weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(acwi, [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], 12)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0.02,-0.4165,-0.3768,-0.3614,-0.3572,-0.3549,-0.3539,-0.3563,-0.3529,-0.3541,-0.3559,-0.3562,-0.3559,-0.3565,-0.3546
0.05,-0.329,-0.2813,-0.2669,-0.2624,-0.2608,-0.2594,-0.2636,-0.259,-0.2589,-0.26,-0.261,-0.2606,-0.2596,-0.2615
0.25,-0.1175,-0.0562,-0.0382,-0.0343,-0.0326,-0.0323,-0.0335,-0.0347,-0.033,-0.0333,-0.0344,-0.0334,-0.0326,-0.0325
0.5,0.0224,0.0779,0.0874,0.0912,0.0917,0.091,0.0908,0.0899,0.0911,0.091,0.0906,0.0908,0.091,0.0911
0.75,0.1464,0.1744,0.1783,0.1815,0.1806,0.1806,0.1806,0.1799,0.1814,0.1808,0.1805,0.1803,0.1802,0.181
0.95,0.2986,0.2955,0.2954,0.295,0.295,0.2935,0.2956,0.2931,0.295,0.2945,0.2952,0.2945,0.2937,0.2932
0.98,0.3618,0.3498,0.3485,0.347,0.3463,0.3464,0.3486,0.3456,0.3486,0.3464,0.3475,0.3483,0.3452,0.346


In [10]:
display("mean")
display(mean(annualise(acwi[:,60:72],12)))
display("std")
display(std(annualise(acwi[:,60:72],12)))
display("skewness")
display(skewness(annualise(acwi[:,60:72],12)))
display("kurtosis")
display(kurtosis(annualise(acwi[:,60:72],12)))

"mean"

0.06311478501465455

"std"

0.16972004257514808

"skewness"

-0.7135443517974893

"kurtosis"

0.6358447495651003

In [11]:
ACWI_period_returns = returns_usd[:ACWI_Net] |> x -> from(x, Date(1998, 01, 01 )) |> x -> to(x, Date(2019,12,31))
returns_summarystats(ACWI_period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1562,1.339,0.0595,0.3651,-2.65,-0.9712,-0.2175,0.122,2.553,0.381,0.1197


In [12]:
M = [1 2 1; 3 4 1; 5 6 1]
cov(M')

3×3 Matrix{Float64}:
 0.333333  0.666667  1.0
 0.666667  2.33333   4.0
 1.0       4.0       7.0

In [13]:
M2 = [1 2 1; 3 4 1; 5 6 missing]
cov(M2')

MethodError: MethodError: Cannot `convert` an object of type Missing to an object of type Float64
The function `convert` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:126
  convert(::Type{T}, !Matched::AbstractChar) where T<:Number
   @ Base char.jl:185
  convert(::Type{T}, !Matched::Gray) where T<:Real
   @ ColorTypes C:\Users\matsz\.julia\packages\ColorTypes\vpFgh\src\conversions.jl:113
  ...
